# Обработка датасета - токенизация, лемматизация и удаление стоп слов

In [1]:
import re
import swifter
from functools import lru_cache
from typing import List
from ast import literal_eval

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer

from tqdm import TqdmWarning
import warnings
from tqdm.auto import tqdm
tqdm.pandas()
warnings.filterwarnings("ignore", category=TqdmWarning)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
morph = MorphAnalyzer()
stop_words = stopwords.words('russian')
extra_stopwords = ['это', 'который', 'весь', 'свой', 'такой', 'тем', 'чтобы']
stop_words.extend(extra_stopwords)
stop_words = set(stop_words)

In [3]:
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Считывание данных

In [4]:
all_data_text = pd.read_csv('../data/data.csv')

In [5]:
all_data_text.head()

,genre,title,author,link,text
0,миниатюры,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов,https://www.proza.ru/2025/07/19/1660,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,миниатюры,Нас не догонят,Нас не догонят,https://www.proza.ru/2025/07/19/1653,Правая нога словно приросла к педали газа. Стр...
2,миниатюры,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану,https://www.proza.ru/2025/07/19/1644,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,миниатюры,Новая кврейская поговорка,Новая кврейская поговорка,https://www.proza.ru/2025/07/19/1641,"Всё, что дороже, чем бесплатно - дорого!"
4,миниатюры,Космическая Свита Примадонны,Космическая Свита Примадонны,https://www.proza.ru/2025/07/19/1630,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


In [6]:
all_data_text = all_data_text.drop(columns=["genre", "author", "link"])

In [7]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,Нас не догонят,Правая нога словно приросла к педали газа. Стр...
2,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,Новая кврейская поговорка,"Всё, что дороже, чем бесплатно - дорого!"
4,Космическая Свита Примадонны,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


### Удаление пустых текстов

In [8]:
all_data_text = all_data_text.dropna().reset_index(drop=True)

In [9]:
all_data_text.head()

,title,text
0,Скромный Подиум Кристаллов,Скромный Подиум Кристаллов.\n.\nИные\nКамни\nБ...
1,Нас не догонят,Правая нога словно приросла к педали газа. Стр...
2,Снежный шар. 200. Не по плану,Снежный шар. 200. Не по плану…\nСолнце давно у...
3,Новая кврейская поговорка,"Всё, что дороже, чем бесплатно - дорого!"
4,Космическая Свита Примадонны,Космическая Свита Примадонны.\n.\nВокруг\nЗемл...


## Чистка текстов от лишних символов

In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [11]:
all_data_text.text = all_data_text.text.apply(clean_text)

In [13]:
all_data_text.title = all_data_text.title.str.lower().str.strip()

In [14]:
all_data_text.head()

,title,text
0,скромный подиум кристаллов,скромный подиум кристаллов иные камни без прим...
1,нас не догонят,правая нога словно приросла к педали газа стре...
2,снежный шар. 200. не по плану,снежный шар 200 не по плану солнце давно укати...
3,новая кврейская поговорка,всё что дороже чем бесплатно дорого
4,космическая свита примадонны,космическая свита примадонны вокруг земли лета...


## Токенизация

In [15]:
def tokenize(text: str) -> List[str]:
    tokens = word_tokenize(text)

    return tokens

In [16]:
all_data_text.text = all_data_text.text.apply(tokenize)
all_data_text.title = all_data_text.title.apply(tokenize)

In [17]:
all_data_text.head()

,title,text
0,"[скромный, подиум, кристаллов]","[скромный, подиум, кристаллов, иные, камни, бе..."
1,"[нас, не, догонят]","[правая, нога, словно, приросла, к, педали, га..."
2,"[снежный, шар, ., 200., не, по, плану]","[снежный, шар, 200, не, по, плану, солнце, дав..."
3,"[новая, кврейская, поговорка]","[всё, что, дороже, чем, бесплатно, дорого]"
4,"[космическая, свита, примадонны]","[космическая, свита, примадонны, вокруг, земли..."


## Удаление строк с небольшими текстами

In [18]:
all_data_text = all_data_text[all_data_text["text"].apply(len) >= 30]

In [19]:
all_data_text.head()

,title,text
0,"[скромный, подиум, кристаллов]","[скромный, подиум, кристаллов, иные, камни, бе..."
1,"[нас, не, догонят]","[правая, нога, словно, приросла, к, педали, га..."
2,"[снежный, шар, ., 200., не, по, плану]","[снежный, шар, 200, не, по, плану, солнце, дав..."
4,"[космическая, свита, примадонны]","[космическая, свита, примадонны, вокруг, земли..."
5,"[боровая, кантата]","[пасхальное, о, это, божественный, лес, где, с..."


In [22]:
len(all_data_text)

14566

In [21]:
all_data_text = all_data_text.reset_index(drop=True)

## Нормализация

In [24]:
all_data_text.head()

,title,text
0,"[скромный, подиум, кристаллов]","[скромный, подиум, кристаллов, иные, камни, бе..."
1,"[нас, не, догонят]","[правая, нога, словно, приросла, к, педали, га..."
2,"[снежный, шар, ., 200., не, по, плану]","[снежный, шар, 200, не, по, плану, солнце, дав..."
3,"[космическая, свита, примадонны]","[космическая, свита, примадонны, вокруг, земли..."
4,"[боровая, кантата]","[пасхальное, о, это, божественный, лес, где, с..."


In [25]:
@lru_cache(maxsize=10000)
def normalize_word(word: str) -> str:
    return morph.parse(word)[0].normal_form.replace('ё', 'е')

def normalization(text: List[str]) -> List[str]:
    return [normalize_word(word) for word in text]

In [26]:
all_data_text.title = (
    all_data_text.title
    .swifter
    .progress_bar(enable=True, desc="Нормализация названий")
    .apply(normalization)
)

Нормализация названий: 100%|██████████| 14566/14566 [00:00<00:00, 21585.04it/s]


In [27]:
all_data_text.text = (
    all_data_text.text
    .swifter
    .progress_bar(enable=True, desc="Нормализация текстов")
    .apply(normalization)
)

Нормализация текстов: 100%|██████████| 14566/14566 [11:24<00:00, 21.28it/s] 


In [28]:
all_data_text.head()

,title,text
0,"[скромный, подиум, кристалл]","[скромный, подиум, кристалл, иной, камень, без..."
1,"[мы, не, догнать]","[правый, нога, словно, прирасти, к, педаль, га..."
2,"[снежный, шар, ., 200., не, по, план]","[снежный, шар, 200, не, по, план, солнце, давн..."
3,"[космический, свита, примадонна]","[космический, свита, примадонна, вокруг, земля..."
4,"[боровой, кантата]","[пасхальный, о, это, божественный, лес, где, с..."


## Удаление стоп слов

In [30]:
def delete_stop_words(text: List[str]) -> List[str]:
    return [word for word in text if word not in stop_words]

Функция нормализации текста нужна, чтобы удалить стоп-слова, так как в списке они все в нормальной форме

In [31]:
all_data_text.text = (
    all_data_text.text
    .swifter
    .progress_bar(enable=True, desc="Обработка текстов")
    .apply(delete_stop_words)
)

Обработка текстов: 100%|██████████| 14566/14566 [00:01<00:00, 7393.31it/s]


Для ускорения работы была сделана параллельная обработка - использовались все ядра устройства

In [33]:
all_data_text.head()

,title,text
0,"[скромный, подиум, кристалл]","[скромный, подиум, кристалл, иной, камень, при..."
1,"[мы, не, догнать]","[правый, нога, словно, прирасти, педаль, газ, ..."
2,"[снежный, шар, ., 200., не, по, план]","[снежный, шар, 200, план, солнце, давно, укати..."
3,"[космический, свита, примадонна]","[космический, свита, примадонна, вокруг, земля..."
4,"[боровой, кантата]","[пасхальный, божественный, лес, сосна, доверчи..."


In [34]:
all_data_text.text = all_data_text.text.apply(' '.join)
all_data_text.title = all_data_text.title.apply(' '.join)
all_data_text.head()

,title,text
0,скромный подиум кристалл,скромный подиум кристалл иной камень примерка ...
1,мы не догнать,правый нога словно прирасти педаль газ стрелка...
2,снежный шар . 200. не по план,снежный шар 200 план солнце давно укатиться го...
3,космический свита примадонна,космический свита примадонна вокруг земля лета...
4,боровой кантата,пасхальный божественный лес сосна доверчиво во...


Перевод списка токенов в строку нужно, так как pandas списки считывает строкой

In [35]:
len(all_data_text)

14566

In [38]:
all_data_text.to_csv("../data/data_lit_proza.csv", index=False)